In [1]:
import pandas as pd
import numpy as np
import re
import requests
import pickle
from konlpy.tag import Okt
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from transformers import TextClassificationPipeline
from transformers import BertTokenizerFast
from transformers import TFBertForSequenceClassification
from datetime import date

import warnings
warnings.filterwarnings('ignore')

In [4]:
loaded_tokenizer = BertTokenizerFast.from_pretrained('./src/bert', from_pt=True)
loaded_model = TFBertForSequenceClassification.from_pretrained('./src/bert', from_pt=True)
classifier = TextClassificationPipeline(tokenizer=loaded_tokenizer, model=loaded_model,
                                            framework='tf', return_all_scores=True)   

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [5]:
def get_code(symbol):
    krx = pd.read_csv('./src/krx_code.csv')
    krx = krx.set_index('한글 종목약명')
    try:
        code = krx.at[symbol,'단축코드']
        return code
    except:
        print('종목명을 다시 확인해주세요.')
        return 0

def get_today():
    today = date.today().isoformat()
    return today

def get_comment(symbol):
    code = get_code(symbol)
    today = get_today()
    comment_list = []
    raw_comment_list = []
    chk = 1
    i = 1
    while chk:  
        url = f'https://finance.naver.com/item/board.naver?code={code}&page={i}'
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36 Edg/100.0.1185.50'}
        res = requests.get(url, headers = headers)
        bs = BeautifulSoup(res.text, 'html.parser')  
        for j in range(20):
            try:
                root = bs.find('div',{'class':'section inner_sub'}).find_all('tr',{'onmouseover':'mouseOver(this)'})[j].text.split('\n') 
                if today != root[1].split()[0].replace('.','-'):
                    chk = 0
                    break
                if len(root) == 14: # 답글
                    pass      
                elif len(root) == 13: # 기본
                    comment = root[3]
                    raw_comment_list.append(comment)
                    comment = re.sub('\[삭제된 게시물의 답글\]',' ',comment)
                    comment = re.sub('[^가-힣]',' ',comment)
                    comment = re.sub(' +',' ',comment)
                    comment = comment.strip()
                    if comment == '':
                        pass
                    else:
                        comment_list.append(comment)                 
                else: # 에러
                    pass
            except: # 에러
                pass
            print(f'\r{today} 댓글{len(comment_list)}개 크롤링중..',end='')
        i += 1
        if chk == 0:
            break   
    print(f'\r{today} 댓글{len(comment_list)}개 크롤링완료')
    return comment_list, raw_comment_list

def BERT_feargreed(symbol):
    comment_list, raw_comment_list = get_comment(symbol)

#     loaded_tokenizer = BertTokenizerFast.from_pretrained('./src/bert', from_pt=True)
#     loaded_model = TFBertForSequenceClassification.from_pretrained('./src/bert', from_pt=True)
#     classifier = TextClassificationPipeline(tokenizer=loaded_tokenizer, model=loaded_model,
#                                             framework='tf', return_all_scores=True)    
    pred_list=[]
    for i in raw_comment_list[:50]:
        f = classifier(i)[0][0]['score']
        g = classifier(i)[0][1]['score']
        if f >= g:
            pred_list.append(1-f)
        else:
            pred_list.append(g)
        print(f'\rBERT모델 댓글{len(pred_list)}개 분석중..',end='')
    print(f'\r{symbol} BERT 공포탐욕지수: {int(sum(pred_list)/len(pred_list)*100)}%')    
    return comment_list

def konlpy_okt(symbol):
    okt = Okt()
    tag_list = ['Noun','Verb','Adjective','VerbPrefix'] 
    comment_list = BERT_feargreed(symbol)
    print('분석 진행중..',end='')
    tokenized_data = []
    for i in range(len(comment_list)):
        tokenized_sentence = okt.pos(comment_list[i], stem=True) 
        tag_checked_sentence = []
        for j in tokenized_sentence:
            x,y = j
            if y in tag_list:
                tag_checked_sentence.append(x)
        if tag_checked_sentence == []:
            pass
        else:
            tokenized_data.append(tag_checked_sentence)     
    for i in tokenized_data:
        for j in range(len(i)):
            i[j] = "'"+i[j]+"'"
    return tokenized_data
    
def tokenize(symbol):
    with open('./src/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)   
    tokenized_data = konlpy_okt(symbol)
    test = tokenizer.texts_to_sequences(tokenized_data)
    test = pad_sequences(test, maxlen=15)
    return test

def feargreed_index(symbol):
    if get_code(symbol) == 0:
        return    
    model = load_model('./src/model.h5')
    test = tokenize(symbol)
    pred = model.predict(test)
    print(f'\r{symbol} LSTM 공포탐욕지수: {int(pred.mean()*100)}%')

In [6]:
feargreed_index('카카오')

2022-05-25 댓글153개 크롤링완료.
카카오 BERT 공포탐욕지수: 40%
카카오 LSTM 공포탐욕지수: 20%
